In [1]:
import tensorflow as tf

2023-12-10 14:39:49.785522: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 14:39:49.813010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 14:39:49.813030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 14:39:49.813683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 14:39:49.818368: I tensorflow/core/platform/cpu_feature_guar

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.15.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### Load the MNIST Dataset

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

11490434/11490434 [==============================] - 1s 0us/step


#### Building a Sequential model

In [8]:
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout

model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10)
])

2023-12-10 14:48:11.072347: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 14:48:11.072402: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 14:48:11.072414: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 14:48:11.295715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 14:48:11.295757: I external/local_xla/xla/stream_executor

In [9]:
predictions = model(x_train[:1]).numpy()
predictions

2023-12-10 14:52:17.744501: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


array([[-0.2783015 ,  0.33878773,  0.88972586,  0.02007115, -0.3619772 ,
        -0.30697528, -0.2552524 ,  0.31092083,  0.06513424, -0.6043241 ]],
      dtype=float32)

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.07009787, 0.12992816, 0.22540973, 0.09446837, 0.06447107,
        0.06811644, 0.07173233, 0.12635744, 0.09882278, 0.05059586]],
      dtype=float32)

In [12]:
from keras.losses import SparseCategoricalCrossentropy

loss_fn = SparseCategoricalCrossentropy(from_logits=True)

In [13]:
loss_fn(y_train[:1], predictions).numpy()

2.6865368

#### Train and Evaluate Model

In [15]:
model.compile(  optimizer='adam',
                loss=loss_fn,
                metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2023-12-10 14:56:52.160918: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f26355b8b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 14:56:52.160949: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti Laptop GPU, Compute Capability 8.6
2023-12-10 14:56:52.171144: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 14:56:52.192510: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1702249012.258700    5090 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 5s 2ms/step - loss: 0.3034 - accuracy: 0.9115
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1440 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1083 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0899 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0764 - accuracy: 0.9757


In [16]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0780 - accuracy: 0.9769 - 708ms/epoch - 2ms/step


[0.07800337672233582, 0.9768999814987183]

In [17]:
# For a probability model
prob_model = Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [18]:
prob_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.8941052e-07, 1.5064987e-08, 1.4140011e-06, 4.6019844e-04,
        9.4570840e-10, 9.6594142e-08, 3.0103646e-14, 9.9952555e-01,
        2.0840441e-06, 1.0204399e-05],
       [4.6580190e-08, 6.1496382e-04, 9.9937564e-01, 4.1890980e-06,
        3.5122697e-12, 2.2315769e-06, 5.0616813e-07, 3.4326493e-12,
        2.4807139e-06, 9.3819614e-13],
       [6.2432872e-07, 9.9932206e-01, 1.2353732e-04, 2.9608436e-05,
        4.7946618e-05, 1.3239793e-05, 4.0936280e-05, 3.8126399e-04,
        3.9858656e-05, 8.7770405e-07],
       [9.9978834e-01, 5.0953824e-10, 7.1606519e-06, 7.6542484e-08,
        9.8160644e-06, 6.5707836e-06, 5.9894346e-06, 1.6506201e-04,
        3.6130331e-08, 1.6918701e-05],
       [3.5452560e-06, 9.2224095e-10, 4.5076849e-06, 1.7420971e-08,
        9.9932492e-01, 1.7364849e-07, 2.0578470e-06, 2.4039738e-05,
        4.0239001e-06, 6.3673151e-04]], dtype=float32)>